
Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.  이 예제에서 사용되는 코드는 Vertex AI Search에 직접 연동해서 좀더 다양한 기능을 효율적으로 사용하는 예제입니다.



지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [13]:
!pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

#GCP 사용자 인증

In [14]:
from google.colab import auth
auth.authenticate_user()

In [15]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL = "gemini-pro"

search_url = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/hackathon-ds_1709838247983/servingConfigs/default_search:search"


In [16]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

vertexai.init(project=PROJECT_ID, location=REGION)

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()


# Vertex AI Search 검색

아래 코드는 REST API를 통해서 Verext AI Search에 직접 검색 쿼리를 보내는 예제입니다.

검색에 활용되는 패러미터는 ContentSearchSpec 에 따릅니다. 아래 정보 참고하세요.

https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec


특히 ExtractiveContentSpec은 검색 컨텐츠를 좀더 상세하게 얻을수 있는 설정입니다. 아래 정보 참고하세요.

https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec


In [17]:
def retrieve_vertex_ai_search(question, search_url, page_size):

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{
          # "snippetSpec": {"maxSnippetCount": 5,
          #                 },
          # "summarySpec": { "summaryResultCount": 5,
          #                  "includeCitations": True},
          "extractiveContentSpec":{
              "maxExtractiveAnswerCount": 3,
              "maxExtractiveSegmentCount": 2,
              "num_previous_segments" : 1,
              "num_next_segments" : 1,
              "return_extractive_segment_score" : True

              }
      },
      # "queryExpansionSpec":{"condition":"AUTO"}
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text


아래 로직은 검색된 결과를 파싱하는 로직입니다.

In [18]:
def parse_discovery_results(response_text):

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    result_index = 0
    searched_ctx_dic = {}
    # summary_ctx= ""
    # if dict_results.get('summary'):
    #     summary_ctx = dict_results['summary']['summaryText']

    if dict_results.get('results'):

        for result in dict_results['results']:

            #snippets_ctx =""
            answer_ctx =""
            segments_ctx =""

            item = {}

            reference = result['document']['derivedStructData']['link']

            derivedStructData = result['document']['derivedStructData']

            if derivedStructData['extractive_answers']:
                for answer in derivedStructData['extractive_answers']:
                    answer_ctx = answer_ctx + answer['content']

            if derivedStructData.get('extractive_segments'):
                for segment in derivedStructData['extractive_segments']:
                    segments_ctx = segments_ctx + segment['content']

            answer_ctx = answer_ctx.replace("<b>","").replace("</b>","").replace("&quot;","")
            segments_ctx = segments_ctx.replace("<b>","").replace("</b>","").replace("&quot;","")
            #reference__name = reference.rsplit('/', 1)[1].strip()
            reference_link = reference.replace("gs://","https://storage.cloud.google.com/")

            item['answer_ctx']= answer_ctx
            item['segments_ctx']= segments_ctx
            item['reference_link']= reference_link

            searched_ctx_dic["Searched Context "+str(result_index)]= item
            result_index = result_index+1

    return searched_ctx_dic

In [19]:
question = "특별법에서 민간 부문의 양자정보통신기술개발 지원에 대해서 상세하게 설명해주세요."

page_size = 3

searched_ctx = retrieve_vertex_ai_search(question, search_url, page_size)
context = parse_discovery_results(searched_ctx)

print(context)

{
  "results": [
    {
      "id": "03fd289b818fe4b883a046fdae36ea17",
      "document": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/hackathon-ds_1709838247983/branches/0/documents/03fd289b818fe4b883a046fdae36ea17",
        "id": "03fd289b818fe4b883a046fdae36ea17",
        "derivedStructData": {
          "extractive_segments": [
            {
              "previous_segments": [
                {
                  "pageNumber": "10",
                  "content": "법제처\n\n10\n\n국가법령정보센터\n\n정보통신 진흥 및 융합 활성화 등에 관한 특별법"
                }
              ],
              "content": "제27조의3(민간 부문의 양자정보통신기술개발 지원) ① 정부는 민간 부문의 양자정보통신기술에 대한 연구개발투자가\n효율적으로 추진될 수 있도록 행정적ㆍ기술적 지원 등 필요한 지원을 할 수 있다.\n② 정부는 양자정보통신기술을 이용한 기술집약형 중소기업 및 벤처기업과 양자정보통신기술을 이용하여 창업하\n는 기업에 대하여 제1항에 따른 지원시책을 우선적으로 추진할 수 있다.\n[본조신설 2020. 6. 9.]\n제27조의3 삭제 \u003c2023. 10. 31.\u003e\n[시행일: 2024. 11. 1.] 제27조의3\n제27조의4(양자정보통신산업클러스터의 지정 등) ① 과학기술정보통신부장관은 양자정보통신 신기술의 창출 및 확산,\n인력

# Gemini Pro 실행 - Vertex AI Search as a Grounding Service


Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold



In [20]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [21]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=question)

print(f"Prompt : {prompt}")
print(f"답변 : {gemini_pro.invoke(prompt)}")


Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {'Searched Context 0': {'answer_ctx': '법제처 10 국가법령정보센터 정보통신 진흥 및 융합 활성화 등에 관한 특별법 제27조의3(민간 부문의 양자정보통신기술개발 지원) ① 정부는 민간 부문의 양자정보통신기술에 대한 연구개발투자가 효율적으로 추진될 수 있도록 행정적ㆍ기술적 지원 등 필요한 지원을 할 수 있다.양자정보통신기술 관련 국제협력의 증진 8. 그 밖에 양자정보통신기술 진흥 및 활성화를 위하여 필요한 사항 ③ 과학기술정보통신부장관은 제2항 각 호의 사업을 효율적으로 추진하기 위하여 전담기관을 지정할 수 있으며, 필 요한 비용의 전부 또는 일부를 보조할 수 있다.② 국가와 지방자치단체는 민간부문의 창의정신을 존중하고 시장중심의 의사형성이 가능하도록 노력하여야 한다. ③ 국가와 지방자치단체는 정보통신 관련 대기업과 중소기업 및 벤처 간의 상생협력과 조화로운 발전을 위하여 노 력하여야 한다.', 'segments_ctx': '제27조의3(민간 부문의 양자정보통신기술개발 지원) ① 정부는 민간 부문의 양자정보통신기술에 대한 연구개발투자가\n효율적으로 추진될 수 있도록 행정적ㆍ기술적 지원 등 필요한 지원을 할 수 있다.\n② 정부는 양자정보통신기술을 이용한 기술집약형 중소기업 및 벤처기업과 양자정보통신기술을 이용하여 창업하\n는 기업에 대하여 제1항에 따른 지원시책을 우선적으로 추진할 수 있다.\n[본조신설 2020. 6. 9.]\n제27조의3 삭제 <2023. 10. 31.>\n[시행일: 2024. 11. 1.] 제27조의3\n제27조의4(양자정보통신산업클러스터의 지정 등) ① 과학기술정보통신부장관은 양자정보통신 신기술의 창출 및 확산,\n인력 양성, 산업 경쟁력 강화 등을 촉진하기 위해 기업, 연구소, 대학